# ETF Momentum Stratagies
## Paper link
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3468556
## summary
investing in etf, long-term (2~4 years) momentum is efficient and robust.monthly rebalancing

In [1]:
import yfinance as yf
import datetime
import pandas as pd

In [2]:
def generate_link(include_leveraged=False):
    link = f"https://etfdb.com/compare/highest-3-year-returns/"
    if not include_leveraged:
        link = link + "no-leveraged/"
    return link

In [3]:
def sub_year(date, y=0):
    m, d = 0, 0
    date = str(date)[:10]
    ny, nm, nd = [int(x) - y for x, y in zip(date.split("-"), [y, m, d])]
    return pd.to_datetime(f"{ny}-{nm}-{nd}")

In [4]:
def get_delta_price(hist, period=3):
    date = hist.index[-1]
    date = sub_year(date, period)
    old = hist.iloc[list(hist.index).index(date)]
    current = hist.iloc[-1]
    return current["Close"]/old["Close"]

In [5]:
SEED_MONEY = 1178
N = 10
INCLUDE_LEVERAGED = False

In [6]:
link = generate_link(INCLUDE_LEVERAGED)

In [7]:
rank = pd.read_html(link)[0]

In [8]:
top_N = rank.iloc[:N]

In [9]:
total_volume = top_N["Avg Volume"].sum()
weights = []
for i in range(len(top_N.index)):
    df = top_N.iloc[i]
    ticker = yf.Ticker(df["Symbol"])
    hist = ticker.history(period="max")
    try:
        dm = get_delta_price(hist)
    except Exception as e:
        print(f"no data for {df['Symbol']}")
        weights.append(0)
        continue
    weights.append((dm * df["Avg Volume"])/total_volume)

no data for ARKG


In [10]:
top_N["weight(%)"] = weights / sum(weights) * 100
top_N["money"] = weights / sum(weights) * SEED_MONEY

/Users/ych/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/ych/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
top_N

,Symbol,Name,3-Year Return,AUM,Avg Volume,weight(%),money
0,ARKW,ARK Web x.0 ETF,141.14%,"$684,006.96",279297,7.544551,88.874807
1,PALL,Aberdeen Standard Physical Palladium Shares ETF,122.78%,"$249,320.85",62371,1.501091,17.682851
2,ARKK,ARK Innovation ETF,118.01%,"$2,899,854.77",803417,19.767624,232.862609
3,ARKG,ARK Genomic Revolution Multi-Sector ETF,117.94%,"$790,562.56",313519,0.000000,0.000000
4,PTH,Invesco DWA Healthcare Momentum ETF,89.20%,"$271,880.73",40848,0.846175,9.967940
5,PSJ,Invesco Dynamic Software ETF,87.79%,"$398,428.02",54886,1.123939,13.240000
6,IGV,iShares Expanded Tech-Software Sector ET,85.24%,"$3,724,105.30",933029,18.809592,221.576999
7,VGT,Vanguard Information Technology ETF,78.63%,"$26,191,590.51",1361321,26.556243,312.832538
8,IBUY,Amplify Online Retail ETF,74.58%,"$277,280.85",61660,1.164427,13.716951
9,SOXX,iShares PHLX Semiconductor ETF,74.54%,"$2,424,360.42",1189446,22.686359,267.245304
